In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
print("TensorFlow version:", tf.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train_dir = '/content/drive/MyDrive/train/'
valid_dir = '/content/drive/MyDrive/valid/'

In [17]:
# parameters
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
# get images
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    seed=123,
    # shuffle=True,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)
valid_ds = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    seed=123,
    # shuffle=True,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

# train_ds = tf.keras.utils.image_dataset_from_directory(
#     train_dir,
#     seed=123,
#     shuffle=True,
#     validation_split=0.2,
#     subset="training",
#     image_size=(img_height, img_width),
#     batch_size=batch_size,
# )
# valid_ds = tf.keras.utils.image_dataset_from_directory(
#     train_dir,
#     seed=123,
#     shuffle=True,
#     validation_split=0.2,
#     subset="validation",
#     image_size=(img_height, img_width),
#     batch_size=batch_size,
# )

class_names = train_ds.class_names
num_classes = len(class_names)

In [22]:
# INCEPTION MODEL
def inception_model():
  input = tf.keras.Input(shape=(224, 224, 3))
  intermediate = tf.keras.layers.Rescaling(1./255)(input)

  # inception
  left = layers.Conv2D(32, 1, padding='same', activation='relu')(intermediate)
  middle_input = layers.Conv2D(32, 1, padding='same', activation='relu')(intermediate)
  middle = layers.Conv2D(32, 3, padding='same', activation='relu')(middle_input)
  right_input = layers.Conv2D(32, 1, padding='same', activation='relu')(intermediate)
  right_input = layers.Conv2D(32, 3, padding='same', activation='relu')(right_input)
  right = layers.Conv2D(32, 5, padding='same', activation='relu')(right_input)
  intermediate = layers.concatenate([left, middle, right], axis = 3)

  intermediate = tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same")(intermediate)
  intermediate = tf.keras.layers.BatchNormalization()(intermediate)
  intermediate = tf.keras.layers.MaxPooling2D()(intermediate)

  left = layers.Conv2D(64, 1, padding='same', activation='relu')(intermediate)
  middle_input = layers.Conv2D(128, 1, padding='same', activation='relu')(intermediate)
  middle = layers.Conv2D(64, 3, padding='same', activation='relu')(middle_input)
  right_input = layers.Conv2D(256, 1, padding='same', activation='relu')(intermediate)
  right_input = layers.Conv2D(128, 3, padding='same', activation='relu')(right_input)
  right = layers.Conv2D(64, 5, padding='same', activation='relu')(right_input)
  intermediate = layers.concatenate([left, middle, right], axis = 3)

  # intermediate = tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same")(intermediate)
  # intermediate = tf.keras.layers.BatchNormalization()(intermediate)
  # intermediate = tf.keras.layers.MaxPooling2D()(intermediate)

  # end
  intermediate = tf.keras.layers.Flatten()(intermediate)
  # intermediate = tf.keras.layers.Dense(1024)(intermediate)
  # intermediate = tf.keras.layers.BatchNormalization()(intermediate)
  # intermediate = tf.keras.layers.Dropout(0.5)(intermediate)
  output = tf.keras.layers.Dense(11)(intermediate)

  # model
  model = tf.keras.Model(input, output)
  optimizer = tf.keras.optimizers.SGD()
  optimizer.momentum = 0.75
  model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

  epochs = 20
  history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=epochs
  )

  # plotting
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs_range = range(epochs)
  plt.figure(figsize=(8, 8))
  plt.subplot(1, 2, 1)
  plt.plot(epochs_range, acc, label='Training Accuracy')
  plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')
  plt.subplot(1, 2, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')
  plt.show()
# inception_model()

In [23]:
def residual(input):
  output = tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same")(input)
  output = tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same")(output)
  out = tf.keras.layers.Add()([input, output])
  return out

def residual2(input):
  output = tf.keras.layers.Conv2D(256, 3, activation="relu", padding="same")(input)
  output = tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same")(output)
  out = tf.keras.layers.Add()([input, output])
  return out

In [24]:
# RESIDUAL CONNECTIONS MODEL
# Background knowledge and inspiration taken from https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba

def residual_model():
  # starting
  input = tf.keras.Input(shape=(224, 224, 3))
  intermediate = tf.keras.layers.Rescaling(1./255)(input)
  intermediate = tf.keras.layers.Conv2D(32, 3, activation="relu", padding="same")(intermediate)
  intermediate = tf.keras.layers.BatchNormalization()(intermediate)
  intermediate = tf.keras.layers.MaxPooling2D()(intermediate)
  intermediate = tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same")(intermediate)
  intermediate = tf.keras.layers.BatchNormalization()(intermediate)
  intermediate = tf.keras.layers.MaxPooling2D()(intermediate)

  # residual
  intermediate = residual(intermediate)
  intermediate = tf.keras.layers.BatchNormalization()(intermediate)
  intermediate = tf.keras.layers.MaxPooling2D()(intermediate)
  intermediate = tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same")(intermediate)
  intermediate = tf.keras.layers.BatchNormalization()(intermediate)
  intermediate = tf.keras.layers.MaxPooling2D()(intermediate)
  intermediate = residual2(intermediate)
  intermediate = tf.keras.layers.BatchNormalization()(intermediate)
  intermediate = tf.keras.layers.MaxPooling2D()(intermediate)

  # end
  intermediate = tf.keras.layers.Flatten()(intermediate)
  intermediate = tf.keras.layers.Dense(1024, activation='relu')(intermediate)
  intermediate = tf.keras.layers.BatchNormalization()(intermediate)
  intermediate = tf.keras.layers.Dropout(0.5)(intermediate)
  output = tf.keras.layers.Dense(11)(intermediate)

  # model
  model = tf.keras.Model(input, output)
  optimizer = tf.keras.optimizers.SGD()
  optimizer.momentum = 0.75
  model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

  epochs = 20
  history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=epochs
  )

  # plotting
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs_range = range(epochs)
  plt.figure(figsize=(8, 8))
  plt.subplot(1, 2, 1)
  plt.plot(epochs_range, acc, label='Training Accuracy')
  plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')
  plt.subplot(1, 2, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')
  plt.show()
# residual_model()

In [25]:
# BEST MODEL
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)
num_classes = 11
initializer = tf.keras.initializers.GlorotNormal(seed=123)
model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(128, 3, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(256, 3, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(512, 3, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(2048, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1024, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(num_classes, kernel_initializer=initializer),
])
optimizer = tf.keras.optimizers.SGD()
optimizer.momentum = 0.75
model.compile(
  optimizer=optimizer,
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
# train model
epochs = 20
history = model.fit(
  train_ds,
  validation_data=valid_ds,
  epochs=epochs
)
# plotting
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
print('\n--- SUMMARY ---')
print("Max Acc - Train:", max(acc), " Valid:", max(val_acc))
print("Min Loss - Train:", min(loss), " Valid:", min(val_loss))

# plotting
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# save model
checkpoint_path = "/content/drive/MyDrive/savedmodel/training_model.ckpt"
# Create a callback that saves the model's weights for best model (best = highest train accuracy)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_best_model=True)

# data augmentation
data_augmentation = tf.keras.Sequential([
  # layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  # layers.RandomContrast(factor=0.05, seed=123),
])
train_ds_1 = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
history = model.fit(
  train_ds_1,
  validation_data=valid_ds,
  epochs=epochs
)
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
print('\n--- SUMMARY ---')
print("Max Acc - Train:", max(acc), " Valid:", max(val_acc))
print("Min Loss - Train:", min(loss), " Valid:", min(val_loss))

# data augmentation
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  # layers.RandomContrast(factor=0.05, seed=123),
])
train_ds_1 = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
history = model.fit(
  train_ds_1,
  validation_data=valid_ds,
  epochs=epochs
)
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
print('\n--- SUMMARY ---')
print("Max Acc - Train:", max(acc), " Valid:", max(val_acc))
print("Min Loss - Train:", min(loss), " Valid:", min(val_loss))
    
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomContrast(factor=0.05, seed=123),
])
train_ds_1 = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
history = model.fit(
  train_ds_1,
  validation_data=valid_ds,
  epochs=epochs
)
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
print('\n--- SUMMARY ---')
print("Max Acc - Train:", max(acc), " Valid:", max(val_acc))
print("Min Loss - Train:", min(loss), " Valid:", min(val_loss))

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomContrast(factor=0.1, seed=123),
])
# SAVE WEIGHTS
train_ds_2 = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
history = model.fit(
  train_ds_2,
  validation_data=valid_ds,
  epochs=epochs,
  callbacks=[cp_callback]
)
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
print('\n--- SUMMARY ---')
print("Max Acc - Train:", max(acc), " Valid:", max(val_acc))
print("Min Loss - Train:", min(loss), " Valid:", min(val_loss))

# plotting
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [28]:
### --- TESTING --- ###

In [ ]:
# --- TEST SCRIPT CELL ---
import json

# model
num_classes = 11
class_names = ['creamy_paste', 'diced', 'floured', 'grated', 'juiced', 'jullienne', 'mixed', 'other', 'peeled', 'sliced', 'whole']
initializer = tf.keras.initializers.GlorotNormal(seed=123)

# MODEL
model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(128, 3, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(256, 3, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(512, 3, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(2048, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1024, activation='relu', kernel_initializer=initializer),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(num_classes, kernel_initializer=initializer),
])
optimizer = tf.keras.optimizers.SGD()
optimizer.momentum = 0.75
model.compile(
  optimizer=optimizer,
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

# load model weights
checkpoint_path = "/content/drive/MyDrive/savedmodel/training_model.ckpt"
model.load_weights(checkpoint_path)



# testing
dictionary = {}
img_height = 224
img_width = 224
list_images = glob.glob("/content/drive/MyDrive/test/anonymous/*.jpg")
for image_path in list_images:
    items = image_path.split('/')
    image_name = items[-1]
    img = tf.keras.utils.load_img(
        path=image_path, target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    prediction_class = class_names[np.argmax(score)]
    dictionary[image_name] = prediction_class

# Writing to prediction.json file
json_object = json.dumps(dictionary, indent=4)
with open("/content/drive/MyDrive/prediction.json", "w+") as outfile:
    outfile.write(json_object)